# Klassifikation 

In diesem Notebook trainieren wir einen Classifier. Aus Beispielen lernt der Classifier, zu welcher Klasse ein Element gehört, und kann neue Elemente mit den gelernten Regeln einordnen. 

Wir lesen zunächst Dokumente aus 4 Wikipedia-Kategorien ein.

### Texten sammeln (API)

In [1]:
import SaveWiki

#SaveWiki.downloadWikiCat('Bösartige Tumorbildung','krebs')
#SaveWiki.downloadWikiCat('Krankheitsbild in der Kardiologie','kardiologie')

##### Texten aufmachen und lesen

In [2]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [3]:
bsptext= readtext('krebs/Adenokarzinom.txt')
print(bsptext[:300])

Als Adenokarzinom bezeichnet man einen bösartigen (malignen) von der Deckzellschicht (Epithel) ausgehenden Tumor, der aus Drüsengewebe hervorgegangen ist. Die gutartige (benigne) Zellveränderung von Drüsengewebe nennt man dagegen Adenom.
Adenokarzinome kommen vor allem im Bereich der Verdauungsorgan


### Merkmalen des Textes auswählen

***stopwords ausschließen***

***Zeichen ausschließen***

***closed class words ausschließen***

In [4]:
#stopwords

In [5]:
import nltk
import re
from HanTa import HanoverTagger as ht
from collections import Counter

stopwords = nltk.corpus.stopwords.words('german')
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

def closed_class(pos):
    if pos[0] == '$':
        return True
    elif pos in ["APPR", "APPRART", "APPO", "APZR", "ART", "KOUI", "KOUS", "KON", "KOKOM", "PDS", "PDAT", "PIS", "PIAT", "PIDAT", "PPER", "PPOSS", "PPOSAT", "PRELS", "PRELAT", "PRF", "PWS", "PWAT", "PWAV", "PAV", "PTKZU", "PTKNEG", "VAFIN", "VAIMP", "VAINF", "VAPP", "VMFIN", "VMINF", "VMPP"]:
        return True
    
    return False

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='german')
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='german')
        tokens = [lemma for (word,lemma,pos) in tagger.tag_sent(tokens) if not closed_class(pos)]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)

    return {w:wordcounts[w]/tlen for w in wordcounts}

Wir versuchen aus jede Klasse 50 Dokumente zu lesen, die nicht extrem kurz oder lang sind.

In [6]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        dirsize = 0
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            if len(text) > 500 and len(text) < 10000:
                docs.append((features_from_text(text),directory))
                dirsize += 1
            if dirsize >= 50:
                break
    return docs

data = read_data(['infekt','krebs','pneumologie','kardiologie'])

In [7]:
data[1]

({'akut': 0.0273972602739726,
  'eitrig': 0.0319634703196347,
  'Thyreoiditis': 0.0365296803652968,
  'handeln': 0.0045662100456621,
  'selten': 0.0091324200913242,
  'schmerzhaft': 0.0045662100456621,
  'Entzündung': 0.0136986301369863,
  'Schilddrüse': 0.0228310502283105,
  'meist': 0.0045662100456621,
  'Infektion': 0.0228310502283105,
  'Bakterie': 0.0091324200913242,
  'zurückführen': 0.0045662100456621,
  'sehen': 0.0045662100456621,
  'infektiöse': 0.0045662100456621,
  'thyreoiditis': 0.0091324200913242,
  'Ursache': 0.0045662100456621,
  'Krankheitsentstehung': 0.0045662100456621,
  'häufig': 0.0091324200913242,
  'Erreger': 0.0045662100456621,
  'grampositiv': 0.0045662100456621,
  'streptococcus': 0.0091324200913242,
  'pyogenes': 0.0045662100456621,
  'pneumoniae': 0.0045662100456621,
  'staphylococcus': 0.0045662100456621,
  'aureus': 0.0045662100456621,
  'gelingen': 0.0045662100456621,
  'Regel': 0.0045662100456621,
  'Lymphgefäße': 0.0045662100456621,
  'Schilddrüsengew

Wir mischen die Daten und teilen in Test- und Trainingsdaten

In [9]:
len(data)

200

In [10]:
import random

random.shuffle(data)
train_data = data[:160]
test_data = data[160:]

Wir schauen uns jetzt mal ein Dokument an:

In [11]:
len(train_data[27])

2

Wir haben jetzt für jedes Dokument einene Merkmalsvektor. Man merke: der Vektor ist eigentlich so lang wie die Anzahl der unterschiedlichen Wörter in der ganzen Sammlung. Alle Wörter, die nicht erwähnt werden haben den Wert 0. 

Wörter, die nur in ein oder zwei Dokumementen vorkommen sind, für die Klassifikation nicht besonders nützlich. 
Wir nutzen nachher nur die Wörter, die in mindestens 5 Dokumente vorkommen. Um das vorzubereiten, berechnen wir für alle Wörter die Dokumentfrequenz>

In [12]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [13]:
#docfreq

## Klassifikation mit Scikit Learn

Die Bibliothek Scikit Learn stellt verschiedene Klassifikationsmodelle zur Verfügung. Wir müssen jetzt richtige Merkmalsvektoren aufbauen, bei denen die Position die Bedeutung einer Zahl bestimmt. Dazu machen wir erst eine feste Liste mit allen Wörtern. Wörter, die zu selten sind lassen wir weg.

In [14]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 4]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

Wir schauen uns mal einen Vektor an:

In [15]:
len(train_vec[135])

1196

In [16]:
train_label[137]

'kardiologie'

Ein einfaches, aber effektives Klassifikationsmodell, das meistens gute Ergebnisse liefert, ist die logistische Regression.

In [17]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\r71-dn9-u1\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=1000000000.0, verbose=True)

Wir können jetzt mit dem trainierten Classifier ein neues Dokument klassifizieren:

In [18]:
v = make_feat_vec(test_data[17][0],allfeatures) 
logreg.predict([v])

array(['kardiologie'], dtype='<U11')

Richtig ist:

In [19]:
test_data[17][1]

'kardiologie'

Oder direkt für alles Testdaten:

In [20]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [21]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

82.5 Prozent korrekt


### Evaluation

f-Faktor
Recall
Precision
Stearman

Wo sind aber die Fehler aufgetreten? Un diese Frage zu beantworten, ist sogenannte eine _Konfusionsmatrix_ sehr hilfreich. NLTK hat eine Funktion zum erstellen eines Konfusionsmatrizen:

In [22]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

            |     k     p |
            |     a     n |
            |     r     e |
            |     d     u |
            |     i     m |
            |  i  o     o |
            |  n  l  k  l |
            |  f  o  r  o |
            |  e  g  e  g |
            |  k  i  b  i |
            |  t  e  s  e |
------------+-------------+
     infekt | <6> .  2  1 |
kardiologie |  . <9> .  . |
      krebs |  .  .<13> . |
pneumologie |  .  3  1 <5>|
------------+-------------+
(row = reference; col = test)



Es gibt verschiedene andere Klassifikatorn, die wie nutzen können:

In [23]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_vec,train_label)
pred_label = list(knn.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

65.0 Prozent korrekt


In [24]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(train_vec,train_label)
pred_label = list(rf.predict(test_vec))
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

82.5 Prozent korrekt
